In [1]:
# Imports
import os
import subprocess
import sys

# REMOVE!!
from ir_utils import *

# model

from ir_lmart import *

# HPO

from hpo import *
# from HpoWorker import *
from HpoWorker_CV import *

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

# from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.WARNING)

# HPO server and stuff

# import logging
# logging.basicConfig(level=logging.WARNING)

import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.optimizers import RandomSearch as RS

import random

# In[2]:


# In[3]:

In [2]:
def tickets(max):
    return {i:i for i in range(1,max)}

In [3]:
million_tickets = tickets(1000000)

In [4]:
len(million_tickets)

999999

In [5]:
def get_train_budget_data_file(budget, query_list, train_data_file):
    # Budget is percentage of training data: 
    # min_budget = 10%
    # max_budget = 100%
    if (int(budget) <= 100 or int(budget) >= 10):
        len_queries = len(query_list)
        budgeted_queries = round(len_queries * (budget / 100))
        print('total budget:', len_queries)
        print('allocated budget:', budgeted_queries)
        train_budget_queries_file = train_data_file + '_budget' + str(budget)
        if not os.path.exists(train_budget_queries_file):
            with open(train_data_file, 'rt') as f_in:
                with open(train_budget_queries_file, 'wt') as budget_file_out:
                    for query_feature in f_in:                        
                        qid = query_feature.split()[1].split(':')[1]
#                         print(qid)
                        if qid in query_list[0:budgeted_queries]:
                            
                            budget_file_out.write(query_feature)
        else:
            print("File already exists")
            return train_budget_queries_file                
    else:
        print('Budget is outside the limits (10% < b < 100%): ', budget)
        return 

In [6]:
# Functions
def generate_run_file(pre_run_file, run_file):
    
    with open(pre_run_file, 'rt') as input_f:
        pre_run = input_f.readlines()
#         print(type(pre_run))
    with open(run_file, 'wt') as out_f:
        for line in pre_run:
            out_f.write(line.replace('docid=','').replace('indri', 'lambdaMART'))
        
# In[4]:

In [7]:
class fakeParser:
    def __init__(self):
        self.dataset = 'bioasq' 
        self.min_budget = 10
        self.max_budget = 10
        self.n_iterations = 1
        self.n_workers = 1
        self.hpo_method = 'rs'
        
        

In [8]:
# Main
if __name__ == "__main__":
    
    # Options and variables
    
    parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
    parser.add_argument('--min_budget',   type=int, help='Minimum (percentage) budget used during the optimization.',    default=10)
    parser.add_argument('--max_budget',   type=int, help='Maximum (percentage) budget used during the optimization.',    default=100)
    parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=500)
    parser.add_argument('--n_workers', type=int,   help='Number of workers to run in parallel.', default=5)
    parser.add_argument('--dataset',   type=str, help='')
#     parser.add_argument('--fold', type=str,   help='')
    parser.add_argument('--hpo_method',   type=str, help='')

#     args=parser.parse_args()
    args = fakeParser()
    
    
    hpo_method = args.hpo_method


    dataset = args.dataset
    workdir = './' + dataset + '_dir/'
#     data_split =  'train'
    ranklib_location = '../../../ranklib/'
    trec_eval_command = '../../eval/trec_eval'
    

    
    metric2t = 'MAP' # 'MAP, NDCG@k, DCG@k, P@k, RR@k, ERR@k (default=ERR@10)'
    
    ranker_type = '6' # LambdaMART
    
    # normalization: Feature Engineering?
    norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'
    
    
    if hpo_method == 'rs':
        hpo_run_id = "RandomSearch"
        # Start a nameserver (see example_3)
        NS = hpns.NameServer(run_id = hpo_run_id, host='127.0.0.1', port=None)
        NS.start()
    elif hpo_method == 'bohb':
        hpo_run_id = "BOHB"
        NS = hpns.NameServer(run_id = hpo_run_id, host='127.0.0.1', port=None)
        NS.start()
    

In [9]:
    workers=[]
    for i in range(args.n_workers):
        
        
        worker = HpoWorker(dataset, workdir, ranklib_location, norm_params, ranker_type,trec_eval_command, metric2t, million_tickets, nameserver='127.0.0.1',run_id=hpo_run_id, id=i)
        worker.run(background=True)
        workers.append(worker)


        # In[13]:

In [10]:
# # Run an optimizer (see example_2)


In [11]:
# Random search

if hpo_method == 'rs':
    rs = RS(  configspace = worker.get_configspace(),
                          run_id = hpo_run_id, 
                          min_budget = args.max_budget, max_budget = args.max_budget
                   )
    res = rs.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)

    rs.shutdown(shutdown_workers=True)
elif hpo_method == 'bohb':
    bohb = BOHB(  configspace = worker.get_configspace(),
                          run_id = hpo_run_id, 
                          min_budget = args.min_budget, max_budget = args.max_budget
                   )
    res = bohb.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)
    bohb.shutdown(shutdown_workers=True)


# In[14]:


# Step 4: Shutdown
# After the optimizer run, we must shutdown the master and the nameserver.



total budget: 1751
allocated budget: 175
File already exists
['java', '-jar', '../../../ranklib/RankLib-2.12.jar', '-train', './bioasq_dir/bioasq_train_features_budget175', '-norm', 'zscore', '-validate', './bioasq_dir/bioasq_dev_features', '-ranker', '6', '-metric2t', 'MAP', '-feature', './bioasq_config/bioasq_lmart_enabled_features', '-leaf', '60', '-shrinkage', '0.46', '-tree', '100', '-save', './bioasq_dir/bioasq_lmart_id1_budget10_leaves60_lr0.46_n100']
Model saved:  ./bioasq_dir/bioasq_lmart_id1_budget10_leaves60_lr0.46_n100
['../../eval/trec_eval', '-m', 'map', '-m', 'P.20', '-m', 'ndcg_cut.20', './bioasq_dir/bioasq_dev_qrels', './bioasq_dir/run_bioasq_lmart_id1_budget10_leaves60_lr0.46_n100']
Run error:  None


In [12]:

NS.shutdown()


# In[15]:


# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()


# In[16]:


all_runs = res.get_all_runs()



print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in all_runs])/args.max_budget))
print('The run took  %.1f seconds to complete.'%(all_runs[-1].time_stamps['finished'] - all_runs[0].time_stamps['started']))


# In[18]:


#     qrels_test_file = './bioasq_dir/bioasq.test_qrels'
#     run_val_file = './this.file'
#     lmart_model = res['info']
#     lmart_model.gen_run_file(test_data_file, run_val_file)
#     eval(trec_eval_command, qrels_test_file, run_val_file)


# In[10]:

Best found configuration: {'learning_rate': 0.46, 'n_leaves': 60, 'n_trees': 100}
A total of 1 unique configurations where sampled.
A total of 1 runs where executed.
Total budget corresponds to 1.0 full function evaluations.
The run took  10.2 seconds to complete.


In [13]:
res.get_incumbent_id()

(0, 0, 0)

In [14]:
id2config[res.get_incumbent_id()]

{'config': {'learning_rate': 0.46, 'n_leaves': 60, 'n_trees': 100},
 'config_info': {}}

In [15]:
# res.get_learning_curves()

In [16]:
res.get_all_runs()

[config_id: (0, 0, 0)	budget: 10.000000	loss: 0.3883
 time_stamps: 0.0013155937194824219 (submitted), 0.0014717578887939453 (started), 10.248464822769165 (finished)
 info: {'s': {'loss': 0.6117, 'info': {'map': '0.3883', 'P_20': '0.2270', 'ndcg_cut_20': '0.4883'}}, 'mean_loss': 0.6117, 'std_loss': 0.0}]

In [17]:
res.get_id2config_mapping()

{(0, 0, 0): {'config': {'learning_rate': 0.46, 'n_leaves': 60, 'n_trees': 100},
  'config_info': {}}}